In [9]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from rouge_score import rouge_scorer
from collections import defaultdict
import numpy as np

In [16]:
data = load_dataset('neural-bridge/rag-dataset-1200')

In [18]:
df = pd.DataFrame()
df['context'] = data['test']['context']
df['question'] = data['test']['question']
df['answer'] = data['test']['answer']
df['id'] = np.arange(len(df))
df['id'] = df['id'].astype(str)

In [58]:
text_splitter = RecursiveCharacterTextSplitter(separators=[" ", ",", "\n"], chunk_size=500, chunk_overlap=0, length_function=len)

In [23]:
ids, splited_doc = [], []
cnt = 0
for text, idx in zip(df['context'].to_numpy(), df['id'].to_numpy()):
    split = text_splitter.split_text(text)
    splited_doc.extend(split)
    ids.extend(list(map(str, np.arange(cnt, cnt+len(split)))))
    cnt+=len(split)
    

In [24]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="rag_project")


In [25]:
collection.add(
    documents=splited_doc,
    ids=ids
)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("meta-llama/llama-3-8b", load_in_4bit=True, device_map="auto")

model = AutoModelForCausalLM.from_pretrained("meta-llama/llama-3-8b", pad_token_id=tokenizer.eos_token_id).to(torch_device)


In [62]:
rag_answer = []

In [65]:
for full_context, question in zip(df['context'].to_numpy(), df['question'].to_numpy()):
    
    results = collection.query(
        query_texts=[question], 
        n_results=1 
    )
    
    prompt = f'''Using the information contained in the context, give a comprehensive answer to the question. Respond only to the question asked, response should be concise and relevant to the question.Provide the number of the source document when relevant.If the answer cannot be deduced from the context, do not give an answer.
    Context:{results['documents'][0]}
    Now here is the question you need to answer.
    Question: {question}
    Answer: '''
    
    model_inputs = tokenizer(prompt, return_tensors='pt').to(torch_device)

  
    greedy_output = model.generate(**model_inputs, max_new_tokens=200)
    
    rag_answer.append(tokenizer.decode(greedy_output[0], skip_special_tokens=True))
    
    print('1')
    

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [31]:
def get_rouge_score(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])
    aggregate_score = defaultdict(list)

    for result, ref in zip(predictions, references):
        for key, val in scorer.score(result, ref).items():
            aggregate_score[key].append(val.fmeasure)

    scores = {}
    for key in aggregate_score:
        scores[key] = np.mean(aggregate_score[key])
    
    return scores

In [56]:
get_rouge_score(rag_answer, df['answer'].to_numpy())

{'rouge1': 0.2514650332232897,
 'rouge2': 0.16353398546257392,
 'rougeL': 0.21479818476285897,
 'rougeLsum': 0.22895632249333434}